Install the required libraries

In [ ]:
!pip3 install dash plotly
!pip3 install dash-bootstrap-components
!pip3 install pandas

Load the required libraries

In [ ]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import pandas as pd

The main code is here

In [ ]:
# Import the full dataframe
df_info_top_250_full = pd.read_csv('df_info_top_250_full.csv')

In [ ]:
# Function to split genres and create new columns Genre 1, Genre 2, Genre 3
def split_genres(row):
    genres = row['Genre'].split(',')
    genre1 = genres[0] if len(genres) >= 1 else None
    genre2 = genres[1] if len(genres) >= 2 else None
    genre3 = genres[2] if len(genres) >= 3 else None
    return genre1, genre2, genre3

df_info_top_250_full[['Genre 1', 'Genre 2', 'Genre 3']] = df_info_top_250_full.apply(split_genres, axis=1, result_type='expand')

In [ ]:
df_info_top_250_full

In [ ]:
# Load the dataset with the reviews with stopwords
df_reviews_top_250 = pd.read_csv('IMDB_Reviews_Top_250_preprocessed.csv')

In [ ]:
# Set the threshold for binary conversion
threshold = 7.0

# Convert the 'Rating' column to a binary variable
df_reviews_top_250['Binary Rating'] = df_reviews_top_250['Rating'].apply(lambda x: "Positive" if x >= threshold else "Negative")

In [ ]:
# Function to count the number of words in a given text
def count_words(text):
    words = text.split()
    return len(words)

In [ ]:
# Apply the function to count words for each Review Title and Review Text
df_reviews_top_250['Review Title Word Count'] = df_reviews_top_250['Review Title'].apply(count_words)
df_reviews_top_250['Review Text Word Count'] = df_reviews_top_250['Review Text'].apply(count_words)

In [ ]:
df_reviews_top_250

In [ ]:
# Plot movies per year

# Count the movies for each year
year_counts = df_info_top_250_full['Year'].value_counts().reset_index()
year_counts.columns = ['Year', 'Count']

# Sort the DataFrame by year in ascending order
year_counts_sorted = year_counts.sort_values('Year')

# Create the bar chart
fig_movies_per_year = go.Figure()

fig_movies_per_year.add_trace(go.Bar(
    x=year_counts_sorted['Year'],
    y=year_counts_sorted['Count'],
    text=year_counts_sorted['Count'],
    textposition='auto',
))

# Customize the layout
fig_movies_per_year.update_layout(
    title='Movies per Year',
    xaxis=dict(title='Year', showgrid=False),
    yaxis=dict(title='Movie Count', showgrid=False, range=[0, 40]),
    hovermode='closest',
)

# Show the plot
fig_movies_per_year.show()

In [ ]:
# Extract the top five years
top_five_years = df_info_top_250_full['Year'].value_counts().nlargest(5).index

In [ ]:
# Create a list with dbc.ListGroupItem elements for the top five years
list_top_five_years = [
    html.Div([
        dbc.ListGroupItem(html.H4("Years with the most movies (Top 5)", style={'fontWeight': 'bold'}), disabled=True),
        html.Hr(),
    ]),
]+ [
    dbc.ListGroupItem(f"{year}: {count} movies") for year, count in zip(top_five_years, df_info_top_250_full['Year'].value_counts().nlargest(5))
]

In [ ]:
# Extract the top five ratings
top_five_ratings = df_info_top_250_full.nlargest(5, 'Rating')

In [ ]:
# Calculate the average movie rating
average_rating = df_info_top_250_full['Rating'].mean()

In [ ]:
# Create a list with dbc.ListGroupItem elements for the top five ratings
list_top_five_ratings = [
    html.Div([
        dbc.ListGroupItem(html.H4("Top 5 Movies", style={'fontWeight': 'bold'}), disabled=True),
        dbc.ListGroupItem(html.H6("(Based on Reviews' Ratings)", style={'fontWeight': 'bold'}), disabled=True),
        html.Hr(),  # Add an hr element after the "Top Five Ratings" title
    ]),
] + [
    dbc.ListGroupItem(f"{row['Title']}: {row['Rating']:.2f}") for _, row in top_five_ratings.iterrows()
] + [
    html.Div([
        dbc.ListGroupItem("Average Rating", disabled=True, style={'fontWeight': 'bold'}),
        dbc.ListGroupItem(f"{average_rating:.2f}"),
    ]),
]

In [ ]:
# Group the DataFrame by each genre and count the number of movies in each genre
genre1_counts = df_info_top_250_full['Genre 1'].value_counts()
genre2_counts = df_info_top_250_full['Genre 2'].value_counts()
genre3_counts = df_info_top_250_full['Genre 3'].value_counts()

In [ ]:
# Create pie charts for Genre 1, Genre 2, and Genre 3
fig_genre1 = go.Figure(
    data=[go.Pie(labels=genre1_counts.index, values=genre1_counts.values)])

fig_genre1.update_layout(title='Genre 1 Movies')

fig_genre2 = go.Figure(
    data=[go.Pie(labels=genre2_counts.index, values=genre2_counts.values)])

fig_genre2.update_layout(title='Genre 2 Movies')

fig_genre3 = go.Figure(
    data=[go.Pie(labels=genre3_counts.index, values=genre3_counts.values)])

fig_genre3.update_layout(title='Genre 3 Movies')

In [ ]:
# Ratings Distribution of Top 250 Movies
fig_violin_ratings = go.Figure()
fig_violin_ratings.add_trace(go.Violin(
    y=df_info_top_250_full['Rating'],
    name='Ratings',
    box_visible=True,
    meanline_visible=True,
    fillcolor='lightseagreen'  # Customize the color of the violin plot
))

# Customize the layout of the violin plot
fig_violin_ratings.update_layout(
    title_text='Ratings Distribution of Top 250 Movies',
    yaxis_title='Ratings',
    violinmode='overlay',  # You can set it to 'group' if you want multiple violin plots side by side
)

In [ ]:
# Create the box plot for a specific movie review ratings
def create_rating_box_plot(selected_movie):
    df_selected_movie = df_reviews_top_250[df_reviews_top_250['Movie Title'] == selected_movie]
    fig_rating = go.Figure()
    fig_rating.add_trace(go.Box(y=df_selected_movie['Rating'], name='Rating'))
    fig_rating.update_layout(
        title='Rating Box Plot',
        yaxis=dict(title='Rating'),
    )
    return fig_rating

In [ ]:
# Count positive and negative reviews
positive_reviews = df_reviews_top_250[df_reviews_top_250['Binary Rating'] == 'Positive'].shape[0]
negative_reviews = df_reviews_top_250[df_reviews_top_250['Binary Rating'] == 'Negative'].shape[0]

# Create a bar chart
fig_binary_reviews_all = go.Figure()

fig_binary_reviews_all.add_trace(go.Bar(
    x=['Positive Reviews', 'Negative Reviews'],
    y=[positive_reviews, negative_reviews],
    text=[positive_reviews, negative_reviews],
    textposition='auto',
    marker=dict(color=['green', 'red']),
))

# Customize the layout
fig_binary_reviews_all.update_layout(
    title='Positive and Negative Reviews',
    xaxis=dict(title='Sentiment'),
    yaxis=dict(title='Number of Reviews'),
)

# Show the plot
fig_binary_reviews_all.show()

In [ ]:
# Word Count Distribution of Review Titles of Top 250 Movies
fig_violin_review_title_word_count_all = go.Figure()
fig_violin_review_title_word_count_all.add_trace(go.Violin(
    y=df_reviews_top_250['Review Title Word Count'],
    name='Word Count',
    box_visible=True,
    meanline_visible=True,
    fillcolor='purple'  # Customize the color of the violin plot
))

# Customize the layout of the violin plot
fig_violin_review_title_word_count_all.update_layout(
    title_text='Word Count Distribution of Review Titles of Top 250 Movies',
    yaxis_title='Review Title Word Count',
    violinmode='overlay',  # You can set it to 'group' if you want multiple violin plots side by side
)

In [ ]:
# Word Count Distribution of Review Text of Top 250 Movies
fig_violin_review_text_word_count_all = go.Figure()
fig_violin_review_text_word_count_all.add_trace(go.Violin(
    y=df_reviews_top_250['Review Text Word Count'],
    name='Word Count',
    box_visible=True,
    meanline_visible=True,
    fillcolor='red'  # Customize the color of the violin plot
))

# Customize the layout of the violin plot
fig_violin_review_text_word_count_all.update_layout(
    title_text='Word Count Distribution of Review Texts of Top 250 Movies',
    yaxis_title='Review Text Word Count',
    violinmode='overlay',  # You can set it to 'group' if you want multiple violin plots side by side
)

In [ ]:
# Calculate the average rating, review title word count, and review text word count for all the movies
average_rating_all = df_reviews_top_250['Rating'].mean()
average_title_word_count_all = df_reviews_top_250['Review Title Word Count'].mean()
average_text_word_count_all = df_reviews_top_250['Review Text Word Count'].mean()

In [ ]:
# Calculate the std of rating, review title word count, and review text word count for all the movies
std_rating_all = df_reviews_top_250['Rating'].std()
std_title_word_count_all = df_reviews_top_250['Review Title Word Count'].std()
std_text_word_count_all = df_reviews_top_250['Review Text Word Count'].std()

In [ ]:
# Step 1: Calculate the average review rating for each movie
average_review_rating = df_reviews_top_250.groupby('Movie Title')['Rating'].mean()

# Step 2: Sort the movies based on the average review rating
sorted_movies = average_review_rating.sort_values(ascending=False)

# Step 3: Select the top 5 movies
top_5_movies_average_review_rating = sorted_movies.head(5)

print(top_5_movies_average_review_rating)

In [ ]:
# Create the list of top 5 movies based on the average review rating
top_movies_list_avg_review_rating = [
    dbc.ListGroupItem(f"{movie}: {word_count:.2f}")
    for movie, word_count in top_5_movies_average_review_rating.items()
]

In [ ]:
# Step 1: Calculate the average review title word count for each movie
average_review_title_word_count = df_reviews_top_250.groupby('Movie Title')['Review Title Word Count'].mean()

# Step 2: Sort the movies based on the average review title word count
sorted_movies = average_review_title_word_count.sort_values(ascending=False)

# Step 3: Select the top 5 movies
top_5_movies_average_review_title_word_count = sorted_movies.head(5)

print(top_5_movies_average_review_title_word_count)

In [ ]:
# Create the list of top 5 movies based on the average review title word count
top_movies_list_avg_review_title = [
    dbc.ListGroupItem(f"{movie}: {word_count:.2f}")
    for movie, word_count in top_5_movies_average_review_title_word_count.items()
]

In [ ]:
# Step 1: Calculate the average review text word count for each movie
average_review_text_word_count = df_reviews_top_250.groupby('Movie Title')['Review Text Word Count'].mean()

# Step 2: Sort the movies based on the average review text word count
sorted_movies = average_review_text_word_count.sort_values(ascending=False)

# Step 3: Select the top 5 movies
top_5_movies_average_review_text_word_count = sorted_movies.head(5)

print(top_5_movies_average_review_text_word_count)

In [ ]:
# Create the list of top 5 movies based on the average review text word count
top_movies_list_avg_review_text = [
    dbc.ListGroupItem(f"{movie}: {word_count:.2f}")
    for movie, word_count in top_5_movies_average_review_text_word_count.items()
]

In [ ]:
# Helper function to calculate various review metrics for the selected movie
def calculate_metrics(selected_movie):
    selected_data = df_reviews_top_250[df_reviews_top_250['Movie Title'] == selected_movie]
    avg_rating = selected_data['Rating'].mean()
    avg_title_word_count = selected_data['Review Title Word Count'].mean()
    avg_text_word_count = selected_data['Review Text Word Count'].mean()
    max_rating = selected_data['Rating'].max()
    max_title_word_count = selected_data['Review Title Word Count'].max()
    max_text_word_count = selected_data['Review Text Word Count'].max()
    min_rating = selected_data['Rating'].min()
    min_title_word_count = selected_data['Review Title Word Count'].min()
    min_text_word_count = selected_data['Review Text Word Count'].min()
    std_rating = selected_data['Rating'].std()
    std_title_word_count = selected_data['Review Title Word Count'].std()
    std_text_word_count = selected_data['Review Text Word Count'].std()
    return (
        avg_rating, avg_title_word_count, avg_text_word_count,
        max_rating, max_title_word_count, max_text_word_count,
        min_rating, min_title_word_count, min_text_word_count,
        std_rating, std_title_word_count, std_text_word_count
    )

In [ ]:
# Create the Dash app and set the Quartz theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.QUARTZ])

In [ ]:
# Create the tabs content for "Movies Overview"
tab1_content = dbc.Container([
    html.H2("Movies Overview", className="text-center"),
    html.Hr(),
    html.Br(),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig_movies_per_year), width=8),
        dbc.Col(dcc.Graph(id='rating-violinplot', figure=fig_violin_ratings), width=4),
    ]),
    html.Br(),
    dbc.Row([
        html.H3("Movies' Leaderboards", className="text-center"),   
        html.Hr(),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col(dbc.ListGroup(list_top_five_years), width=6),
        dbc.Col(dbc.ListGroup(list_top_five_ratings), width=6), 
    ]),
    html.Br(),
    dbc.Row([
        html.H3("Movies' Genre", className="text-center"),
        html.H6("(Each Movie can have up to three genres)", className="text-center"),    
        html.Hr(),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col(html.H3("Genre 1", className="text-center"), width=4),
        dbc.Col(html.H3("Genre 2", className="text-center"), width=4),
        dbc.Col(html.H3("Genre 3", className="text-center"), width=4),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig_genre1), width=4),
        dbc.Col(dcc.Graph(figure=fig_genre2), width=4),
        dbc.Col(dcc.Graph(figure=fig_genre3), width=4),
    ]),
    html.Br(),    
    dbc.Row([
        html.H3("Search for movie details", className="text-center"),   
        html.Hr(),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='movie-dropdown',
            options=[{'label': movie, 'value': movie} for movie in df_info_top_250_full['Title']],
            value=df_info_top_250_full['Title'][0],
            placeholder="Select a movie...",
            style={'color': 'black'}  # Set the text color to black
        ), width=12),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H6("Movie Details", className="text-center"),
                html.P("Position in IMDB: ", id='position-imdb'),
                html.P("Rating: ", id='rating'),
                html.P("Year: ", id='year'),
                html.P("URL: ", id='url'),
            ], style={'background-color': 'rgba(255, 255, 255, 0.2)', 'padding': '10px'}),
        ], width=6),
        dbc.Col([
            dcc.Graph(id='rating-box-plot')
        ], width=6),       
    ]),

], className="pt-4")

In [ ]:
# Create the tabs content for "Reviews Overview"
tab2_content = dbc.Container([
    html.H2("Reviews Overview", className="text-center"),
    html.Hr(),
    html.Br(),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig_binary_reviews_all), width=4),
        dbc.Col(dcc.Graph(figure=fig_violin_review_title_word_count_all), width=4),
        dbc.Col(dcc.Graph(figure=fig_violin_review_text_word_count_all), width=4),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H6("Movie Details", className="text-center"),
                html.P(f"Average Rating: {average_rating_all:.2f}"),
                html.P(f"Average Review Title Word Count: {average_title_word_count_all:.2f}"),
                html.P(f"Average Review Text Word Count: {average_text_word_count_all:.2f}"),
            ], style={'background-color': 'rgba(255, 255, 255, 0.2)', 'padding': '10px'}),
        ], width=6),
        dbc.Col([
            html.Div([
                html.H6("Movie Details", className="text-center"),
                html.P(f"Standard Deviation of Rating: {std_rating_all:.2f}"),
                html.P(f"Standard Deviation of Review Title Word Count: {std_title_word_count_all:.2f}"),
                html.P(f"Standard Deviation of Review Text Word Count: {std_text_word_count_all:.2f}"),
            ], style={'background-color': 'rgba(255, 255, 255, 0.2)', 'padding': '10px'}),
        ], width=6),        
    ]),
    html.Br(),
    dbc.Row([
        html.H3("Movies' Leaderboards based on reviews", className="text-center"),   
        html.Hr(),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([
            html.H5("Top 5 Movies based on Average Review Rating"),
            dbc.ListGroup(top_movies_list_avg_review_rating)
        ], width=4),
        dbc.Col([
            html.H5("Top 5 Movies based on Average Review Title Word Count"),
            dbc.ListGroup(top_movies_list_avg_review_title)
        ], width=4),
        dbc.Col([
            html.H5("Top 5 Movies based on Average Review Text Word Count"),
            dbc.ListGroup(top_movies_list_avg_review_text)
        ], width=4),  
    ]),
    html.Br(),
    dbc.Row([
        html.H3("Search for movie details", className="text-center"),   
        html.Hr(),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col(dcc.Dropdown(id='movie-review-dropdown',
                    options=[{'label': movie, 'value': movie} for movie in df_reviews_top_250['Movie Title'].unique()],
                    value=df_reviews_top_250['Movie Title'].iloc[0],
                    placeholder="Select a movie...",
                    style={'color': 'black'}  # Set the text color to black
               ), width=12),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([
            html.H6("Average Review Movie Details"),
            html.P("Average Rating: ", id='avg-rating'),
            html.P("Average Review Title Word Count: ", id='avg-title-word-count'),
            html.P("Average Review Text Word Count: ", id='avg-text-word-count')
        ], width=3),
        dbc.Col([
            html.H6("Max Review Movie Details"),
            html.P("Max Rating: ", id='max-rating'),
            html.P("Max Review Title Word Count: ", id='max-title-word-count'),
            html.P("Max Review Text Word Count: ", id='max-text-word-count')
        ], width=3),
        dbc.Col([
            html.H6("Min Review Movie Details"),
            html.P("Min Rating: ", id='min-rating'),
            html.P("Min Review Title Word Count: ", id='min-title-word-count'),
            html.P("Min Review Text Word Count: ", id='min-text-word-count'),
        ], width=3),
        dbc.Col([
            html.H6("Std Review Movie Details"),
            html.P("Std Rating: ", id='std-rating'),
            html.P("Std Review Title Word Count: ", id='std-title-word-count'),
            html.P("Std Review Text Word Count: ", id='std-text-word-count'),
        ], width=3),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col(dcc.Graph(id='rating-violin-plot'), width=4),
        dbc.Col(dcc.Graph(id='title-word-count-violin-plot'), width=4),
        dbc.Col(dcc.Graph(id='text-word-count-violin-plot'), width=4),
    ]),

], className="pt-4")

In [ ]:
# Create the layout with two tabs
app.layout = dbc.Container([
    dbc.Navbar(
        dbc.Container([
            html.Img(src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/IMDB_Logo_2016.svg/220px-IMDB_Logo_2016.svg.png", height="30px"),
            dbc.NavbarBrand("IMDB Top 250 Movies Dashboard", className="ml-2")
        ]),
        color="dark",
        dark=True,
    ),
    html.Br(),
    dbc.Tabs(
        [
            dbc.Tab(tab1_content, label="Movies Overview"),
            dbc.Tab(tab2_content, label="Reviews Overview"),
        ]
    ),
    dbc.Container([
        html.P("Created by D. Pantazatos", className="text-center mt-4")
    ], className="mt-auto")
], fluid=True)

In [ ]:
# Callback to update movie details based on the selected movie
@app.callback(
    [
        dash.dependencies.Output('position-imdb', 'children'),
        dash.dependencies.Output('rating', 'children'),
        dash.dependencies.Output('year', 'children'),
        dash.dependencies.Output('url', 'children')
    ],
    [dash.dependencies.Input('movie-dropdown', 'value')]
)

def update_movie_details(selected_movie):
    movie_details = df_info_top_250_full[df_info_top_250_full['Title'] == selected_movie].iloc[0]
    position_imdb = f"Position in IMDB: {movie_details['Position in IMDB Top 250']}"
    rating = f"Rating: {movie_details['Rating']:.2f}"
    year = f"Year: {movie_details['Year']}"
    url = html.A('Link', href=movie_details['Link'], target='_blank')
    return position_imdb, rating, year, url

In [ ]:
# Update the movies reviews ratings box plot when a movie is selected
@app.callback(
    Output('rating-box-plot', 'figure'),
    [Input('movie-dropdown', 'value')]
)
def update_rating_box_plot(selected_movie):
    return create_rating_box_plot(selected_movie)

In [ ]:
# Callback to update movie details based on the selected movie
@app.callback(
    [
        dash.dependencies.Output('avg-rating', 'children'),
        dash.dependencies.Output('avg-title-word-count', 'children'),
        dash.dependencies.Output('avg-text-word-count', 'children'),
        dash.dependencies.Output('max-rating', 'children'),
        dash.dependencies.Output('max-title-word-count', 'children'),
        dash.dependencies.Output('max-text-word-count', 'children'),
        dash.dependencies.Output('min-rating', 'children'),
        dash.dependencies.Output('min-title-word-count', 'children'),
        dash.dependencies.Output('min-text-word-count', 'children'),
        dash.dependencies.Output('std-rating', 'children'),
        dash.dependencies.Output('std-title-word-count', 'children'),
        dash.dependencies.Output('std-text-word-count', 'children'),
        dash.dependencies.Output('rating-violin-plot', 'figure'),
        dash.dependencies.Output('title-word-count-violin-plot', 'figure'),
        dash.dependencies.Output('text-word-count-violin-plot', 'figure')
    ],
    [dash.dependencies.Input('movie-review-dropdown', 'value')]
)

def update_metrics_and_plots(selected_movie):
    avg_rating, avg_title_word_count, avg_text_word_count, \
    max_rating, max_title_word_count, max_text_word_count, \
    min_rating, min_title_word_count, min_text_word_count, \
    std_rating, std_title_word_count, std_text_word_count = calculate_metrics(selected_movie)
    
    # Create violin plots for the selected movie
    rating_violin = go.Violin(y=df_reviews_top_250[df_reviews_top_250['Movie Title'] == selected_movie]['Rating'],
                              box_visible=True, meanline_visible=True, fillcolor='green', line_color='purple',
                              name='Rating', hoverinfo='y', showlegend=False)
    title_word_count_violin = go.Violin(y=df_reviews_top_250[df_reviews_top_250['Movie Title'] == selected_movie]['Review Title Word Count'],
                                        box_visible=True, meanline_visible=True, fillcolor='blue', line_color='purple',
                                        name='Review Title Word Count', hoverinfo='y', showlegend=False)
    text_word_count_violin = go.Violin(y=df_reviews_top_250[df_reviews_top_250['Movie Title'] == selected_movie]['Review Text Word Count'],
                                       box_visible=True, meanline_visible=True, fillcolor='red', line_color='purple',
                                       name='Review Text Word Count', hoverinfo='y', showlegend=False)
    
    rating_violin_fig = go.Figure(rating_violin)
    title_word_count_violin_fig = go.Figure(title_word_count_violin)
    text_word_count_violin_fig = go.Figure(text_word_count_violin)
    
    rating_violin_fig.update_layout(title_text="Rating Violin Plot")
    title_word_count_violin_fig.update_layout(title_text="Review Title Word Count Violin Plot")
    text_word_count_violin_fig.update_layout(title_text="Review Text Word Count Violin Plot")

    # Update the layout with the selected movie's metrics and violin plots
    return (
        f"Movie Rating: {avg_rating:.2f}", f"Review Title Word Count: {avg_title_word_count:.2f}", f"Review Text Word Count: {avg_text_word_count:.2f}",
        f"Movie Rating: {max_rating:.2f}", f"Review Title Word Count: {max_title_word_count:.2f}", f"Review Text Word Count: {max_text_word_count:.2f}",
        f"Movie Rating: {min_rating:.2f}", f"Review Title Word Count: {min_title_word_count:.2f}", f"Review Text Word Count: {min_text_word_count:.2f}",
        f"Movie Rating: {std_rating:.2f}", f"Review Title Word Count: {std_title_word_count:.2f}", f"Review Text Word Count: {std_text_word_count:.2f}",
        rating_violin_fig, title_word_count_violin_fig, text_word_count_violin_fig
    )

In [ ]:
if __name__ == '__main__':
    app.run_server()